# Second Comparison Using LFW Dataset - VGGFace (RESNET50)

Several feature extractor:
1. Face Embedding: Facenet
2. Face Embedding: VGG Face
3. Face Embedding: VGG Face - VGG16
4. Face Embedding: VGG Face - RESNET50
5. LBPH (Local Binary Pattern Histogram)

In [1]:
from scipy.spatial.distance import euclidean
from sklearn.metrics import accuracy_score, f1_score
from keras import backend as K
from feature_extractor.face_feature_extractor import FaceFeatureExtractor
import numpy as np
import cv2 as cv
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import time

Using TensorFlow backend.


# Read Dataset

In [2]:
import os

In [3]:
DIR_PATH = '../lfw/'
image_path_list = []
labels = []
name_dictionary = {}
counter = 0
for root, dirs, files in os.walk(DIR_PATH):
    for filename in files:
        person_name = ' '.join(filename.split('.')[0].split('_')[0:-1]) 
        file_path = os.path.join(root, filename)
        if person_name not in name_dictionary:
            counter += 1
            name_dictionary[person_name] = counter
        image_path_list.append(file_path)
        labels.append(name_dictionary[person_name])

In [4]:
print(len(labels))
print(len(image_path_list))

13233
13233


In [5]:
image_path_list[0:5]

['../lfw/Ryan_Newman/Ryan_Newman_0001.jpg',
 '../lfw/Dimitar_Berbatov/Dimitar_Berbatov_0001.jpg',
 '../lfw/Ed_Rendell/Ed_Rendell_0001.jpg',
 '../lfw/Joe_Crede/Joe_Crede_0001.jpg',
 '../lfw/Norman_Mailer/Norman_Mailer_0001.jpg']

# Experiment

In [6]:
FACENET_MODEL_PATH = 'model_data/facenet/20180402-114759'
VGGFACE_MODEL_PATH = 'model_data/vgg_face_weights.h5'

In [7]:
mtcnn_detector = MTCNN()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
def predict(image_representation_list, true_label, test_representation, min_distance):
    minimum_label = None
    minimum_distance = min_distance
    
    for idx, image_representation in enumerate(image_representation_list):
        distance = euclidean(image_representation, test_representation)
        if distance < minimum_distance:
            minimum_distance = distance
            minimum_label = true_label[idx]
    
    return minimum_label

In [9]:
def compute_score(predictions, labels):
    count_same = 0
    for idx, prediction in enumerate(predictions):
        if labels[idx] == prediction:
            count_same += 1
        elif labels[idx] == -2: # failed to detect face (still measure as valid)
            count_same += 1
    return count_same / len(predictions)

In [10]:
def write_checkpoint(filename, prediction_results):
    f = open(filename, 'w')
    for prediction in prediction_results:
        f.write(str(prediction) + '\n')
    f.close()

## VGGFace RESNET50

Based on experiment VGGFace RESNET50 have distance below 110 for euclidean if two image are the same person

In [11]:
feature_extractor = FaceFeatureExtractor(None, extractor_name='vgg_face_resnet50')

In [12]:
THRESHOLD = 110

### Experiment 1

### Shuffle Dataset Seed=0

In [13]:
import random

In [14]:
temp = list(zip(image_path_list, labels))
random.Random(0).shuffle(temp) # custom seed
image_path_list, labels = zip(*temp)

In [15]:
image_path_list[0:5]

('../lfw/Tommy_Maddox/Tommy_Maddox_0001.jpg',
 '../lfw/David_Millar/David_Millar_0001.jpg',
 '../lfw/Gregg_Popovich/Gregg_Popovich_0004.jpg',
 '../lfw/Shimon_Peres/Shimon_Peres_0001.jpg',
 '../lfw/Rudolph_Giuliani/Rudolph_Giuliani_0024.jpg')

In [16]:
labels[0:5]

(3038, 1517, 3115, 834, 1016)

In [17]:
name_dictionary['Tommy Maddox']

3038

### Experiment Starts

In [18]:
image_representation_database = []
image_representation_labels = []
prediction_result = []

In [19]:
start = time.time()
for idx, image_path in enumerate(image_path_list):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        label = labels[idx]
        if prediction == None: # predict as new person
            if label not in image_representation_labels: # is new person
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # failed to predict (should be old person)
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(-1) 
        else: # predict as old person (always add true label to galery)
            image_representation_labels.append(label)
            image_representation_database.append(feature_test)
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-2)
end = time.time()
print('Time Taken:', end-start)

Checkpoint 0
Checkpoint 1000
Checkpoint 2000
Checkpoint 3000
Checkpoint 4000
Checkpoint 5000
Checkpoint 6000
Checkpoint 7000
Checkpoint 8000
Checkpoint 9000
Checkpoint 10000
Checkpoint 11000
Checkpoint 12000
Checkpoint 13000
Time Taken: 1960.0411639213562


In [20]:
image_path_list[37]

'../lfw/George_W_Bush/George_W_Bush_0233.jpg'

In [21]:
compute_score(prediction_result, labels)

0.6868434973173128

In [22]:
len(prediction_result)

13233

In [23]:
len(labels)

13233

In [24]:
write_checkpoint('VGGFace RESNET LFW Dataset - 1.txt', prediction_result)

### Experiment 2

### Shuffle Dataset Seed=1

In [25]:
import random

In [26]:
temp = list(zip(image_path_list, labels))
random.Random(1).shuffle(temp) # custom seed
image_path_list, labels = zip(*temp)

In [27]:
image_path_list[0:5]

('../lfw/Eddy_Merckx/Eddy_Merckx_0002.jpg',
 '../lfw/Rocco_Buttiglione/Rocco_Buttiglione_0001.jpg',
 '../lfw/Gerhard_Schroeder/Gerhard_Schroeder_0076.jpg',
 '../lfw/Mark_Hogan/Mark_Hogan_0001.jpg',
 '../lfw/Cameron_Diaz/Cameron_Diaz_0003.jpg')

In [28]:
labels[0:5]

(4804, 334, 25, 5684, 4615)

In [29]:
name_dictionary['Eddy Merckx']

4804

### Experiment Starts

In [30]:
image_representation_database = []
image_representation_labels = []
prediction_result = []

In [31]:
start = time.time()
for idx, image_path in enumerate(image_path_list):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        label = labels[idx]
        if prediction == None: # predict as new person
            if label not in image_representation_labels: # is new person
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # failed to predict (should be old person)
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(-1) 
        else: # predict as old person (always add true label to galery)
            image_representation_labels.append(label)
            image_representation_database.append(feature_test)
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-2)
end = time.time()
print('Time Taken:', end-start)

Checkpoint 0
Checkpoint 1000
Checkpoint 2000
Checkpoint 3000
Checkpoint 4000
Checkpoint 5000
Checkpoint 6000
Checkpoint 7000
Checkpoint 8000
Checkpoint 9000
Checkpoint 10000
Checkpoint 11000
Checkpoint 12000
Checkpoint 13000
Time Taken: 1963.902307510376


In [32]:
image_path_list[37]

'../lfw/Tom_Cruise/Tom_Cruise_0007.jpg'

In [33]:
compute_score(prediction_result, labels)

0.6848787123101338

In [34]:
len(prediction_result)

13233

In [35]:
len(labels)

13233

In [36]:
write_checkpoint('VGGFace RESNET LFW Dataset - 2.txt', prediction_result)

### Experiment 3

### Shuffle Dataset Seed=2

In [37]:
import random

In [38]:
temp = list(zip(image_path_list, labels))
random.Random(2).shuffle(temp) # custom seed
image_path_list, labels = zip(*temp)

In [39]:
image_path_list[0:5]

('../lfw/Colin_Powell/Colin_Powell_0014.jpg',
 '../lfw/Sean_OKeefe/Sean_OKeefe_0001.jpg',
 '../lfw/David_Wells/David_Wells_0001.jpg',
 '../lfw/Donald_Rumsfeld/Donald_Rumsfeld_0118.jpg',
 '../lfw/Rainer_Schuettler/Rainer_Schuettler_0003.jpg')

In [40]:
labels[0:5]

(2196, 4787, 5518, 5402, 5235)

In [41]:
name_dictionary['Colin Powell']

2196

### Experiment Starts

In [42]:
image_representation_database = []
image_representation_labels = []
prediction_result = []

In [43]:
start = time.time()
for idx, image_path in enumerate(image_path_list):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        label = labels[idx]
        if prediction == None: # predict as new person
            if label not in image_representation_labels: # is new person
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # failed to predict (should be old person)
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(-1) 
        else: # predict as old person (always add true label to galery)
            image_representation_labels.append(label)
            image_representation_database.append(feature_test)
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-2)
end = time.time()
print('Time Taken:', end-start)

Checkpoint 0
Checkpoint 1000
Checkpoint 2000
Checkpoint 3000
Checkpoint 4000
Checkpoint 5000
Checkpoint 6000
Checkpoint 7000
Checkpoint 8000
Checkpoint 9000
Checkpoint 10000
Checkpoint 11000
Checkpoint 12000
Checkpoint 13000
Time Taken: 1946.1233150959015


In [44]:
image_path_list[37]

'../lfw/Junichiro_Koizumi/Junichiro_Koizumi_0025.jpg'

In [45]:
compute_score(prediction_result, labels)

0.6866923600090682

In [46]:
len(prediction_result)

13233

In [47]:
len(labels)

13233

In [48]:
write_checkpoint('VGGFace RESNET LFW Dataset - 3.txt', prediction_result)

### Experiment 4

### Shuffle Dataset Seed=3

In [49]:
import random

In [50]:
temp = list(zip(image_path_list, labels))
random.Random(3).shuffle(temp) # custom seed
image_path_list, labels = zip(*temp)

In [51]:
image_path_list[0:5]

('../lfw/Lance_Armstrong/Lance_Armstrong_0011.jpg',
 '../lfw/Lleyton_Hewitt/Lleyton_Hewitt_0009.jpg',
 '../lfw/Paul_McCartney/Paul_McCartney_0005.jpg',
 '../lfw/Eva_Dimas/Eva_Dimas_0002.jpg',
 '../lfw/Paul_Murphy/Paul_Murphy_0001.jpg')

In [52]:
labels[0:5]

(4771, 3676, 427, 1004, 5498)

In [53]:
name_dictionary['Lance Armstrong']

4771

### Experiment Starts

In [54]:
image_representation_database = []
image_representation_labels = []
prediction_result = []

In [55]:
start = time.time()
for idx, image_path in enumerate(image_path_list):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        label = labels[idx]
        if prediction == None: # predict as new person
            if label not in image_representation_labels: # is new person
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # failed to predict (should be old person)
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(-1) 
        else: # predict as old person (always add true label to galery)
            image_representation_labels.append(label)
            image_representation_database.append(feature_test)
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-2)
end = time.time()
print('Time Taken:', end-start)

Checkpoint 0
Checkpoint 1000
Checkpoint 2000
Checkpoint 3000
Checkpoint 4000
Checkpoint 5000
Checkpoint 6000
Checkpoint 7000
Checkpoint 8000
Checkpoint 9000
Checkpoint 10000
Checkpoint 11000
Checkpoint 12000
Checkpoint 13000
Time Taken: 1943.3127114772797


In [56]:
image_path_list[37]

'../lfw/Adrian_McPherson/Adrian_McPherson_0002.jpg'

In [57]:
compute_score(prediction_result, labels)

0.6875991838585355

In [58]:
len(prediction_result)

13233

In [59]:
len(labels)

13233

In [60]:
write_checkpoint('VGGFace RESNET LFW Dataset - 4.txt', prediction_result)

### Experiment 5

### Shuffle Dataset Seed=4

In [61]:
import random

In [62]:
temp = list(zip(image_path_list, labels))
random.Random(4).shuffle(temp) # custom seed
image_path_list, labels = zip(*temp)

In [63]:
image_path_list[0:5]

('../lfw/Takashi_Yamamoto/Takashi_Yamamoto_0001.jpg',
 '../lfw/Jacques_Rogge/Jacques_Rogge_0009.jpg',
 '../lfw/Gerhard_Schroeder/Gerhard_Schroeder_0005.jpg',
 '../lfw/Marcus_Garrettson/Marcus_Garrettson_0001.jpg',
 '../lfw/Job_Cohen/Job_Cohen_0001.jpg')

In [64]:
labels[0:5]

(2730, 4713, 25, 598, 1122)

In [65]:
name_dictionary['Takashi Yamamoto']

2730

### Experiment Starts

In [66]:
image_representation_database = []
image_representation_labels = []
prediction_result = []

In [67]:
start = time.time()
for idx, image_path in enumerate(image_path_list):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        label = labels[idx]
        if prediction == None: # predict as new person
            if label not in image_representation_labels: # is new person
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # failed to predict (should be old person)
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(-1) 
        else: # predict as old person (always add true label to galery)
            image_representation_labels.append(label)
            image_representation_database.append(feature_test)
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-2)
end = time.time()
print('Time Taken:', end-start)

Checkpoint 0
Checkpoint 1000
Checkpoint 2000
Checkpoint 3000
Checkpoint 4000
Checkpoint 5000
Checkpoint 6000
Checkpoint 7000
Checkpoint 8000
Checkpoint 9000
Checkpoint 10000
Checkpoint 11000
Checkpoint 12000
Checkpoint 13000
Time Taken: 1950.2563004493713


In [68]:
image_path_list[37]

'../lfw/Paul_Gascoigne/Paul_Gascoigne_0001.jpg'

In [69]:
compute_score(prediction_result, labels)

0.6845764376936446

In [70]:
len(prediction_result)

13233

In [71]:
len(labels)

13233

In [72]:
write_checkpoint('VGGFace RESNET LFW Dataset - 5.txt', prediction_result)